In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import svm
import pandas as pd
import numpy as np
import random

In [3]:
real_fake_file_name="/content/drive/Shareddrives/CSCE_704_Project/fake_or_real_news.csv"
fake_file_name="/content/drive/Shareddrives/CSCE_704_Project/Fake.csv"
real_file_name="/content/drive/Shareddrives/CSCE_704_Project/True.csv"

import csv
dataset=[]

titles=[]
text=[]
labels=[]

with open(real_fake_file_name) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  rows = []
  for row in csvreader:
    titles.append(row[1])
    text.append(row[2])
    labels.append(row[3])

with open(fake_file_name) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  rows = []
  for row in csvreader:
    titles.append(row[0])
    text.append(row[1])
    labels.append("FAKE")

with open(real_file_name) as file:
  csvreader = csv.reader(file)
  header = next(csvreader)
  rows = []
  for row in csvreader:
    titles.append(row[0])
    text.append(row[1])
    labels.append("REAL")

dataset.append(titles)
dataset.append(text)
dataset.append(labels)

In [4]:
# Vect and split

all_text_vectorizer = TfidfVectorizer(stop_words='english')
all_text_vectorized = all_text_vectorizer.fit_transform(dataset[1])

master_data_train, master_data_test, master_labels_train, master_labels_test = train_test_split(all_text_vectorized, dataset[2], test_size=0.6, random_state = 30)

nn_data_train = master_data_test[:25000]
nn_data_test = master_data_test[25000:]

final_labels_train_tf =  master_labels_test[:25000]
final_labels_test_tf =  master_labels_test[25000:]

In [5]:
print(master_data_train.size, master_data_test.size)

3430571 5115737


In [6]:
model_svc = SVC(kernel='linear')
model_svc.fit(master_data_train, master_labels_train)

SVC(kernel='linear')

In [7]:
pred_svc = model_svc.predict(master_data_test)

In [8]:
print(classification_report(master_labels_test, pred_svc))

              precision    recall  f1-score   support

        FAKE       0.96      0.97      0.97     15945
        REAL       0.97      0.96      0.96     14795

    accuracy                           0.97     30740
   macro avg       0.97      0.97      0.97     30740
weighted avg       0.97      0.97      0.97     30740



In [9]:
model_rfc = RandomForestClassifier(n_estimators=200, max_depth=500)
model_rfc.fit(master_data_train, master_labels_train)

RandomForestClassifier(max_depth=500, n_estimators=200)

In [10]:
pred_rfc = model_rfc.predict(master_data_test)

In [11]:
print(classification_report(master_labels_test, pred_rfc))

              precision    recall  f1-score   support

        FAKE       0.94      0.97      0.95     15945
        REAL       0.96      0.94      0.95     14795

    accuracy                           0.95     30740
   macro avg       0.95      0.95      0.95     30740
weighted avg       0.95      0.95      0.95     30740



In [12]:
nn_labels_train = []
for rfc_pred,svc_pred, true_label in zip(pred_rfc[0:25000],pred_svc[0:25000], final_labels_train_tf):
  # rfc_pred = model_rfc.predict(test_data_point)
  # svc_pred = model_svc.predict(test_data_point)

  if rfc_pred == true_label and svc_pred == true_label:
    b = random.randint(0, 1)
    nn_labels_train.append(b)
  elif rfc_pred == true_label and svc_pred != true_label:
    nn_labels_train.append(0)
  elif rfc_pred != true_label and svc_pred == true_label:
    nn_labels_train.append(1)
  else:
    b = random.randint(0, 1)
    nn_labels_train.append(b)

In [13]:
from sklearn.neural_network import MLPClassifier
model_nn1 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 25, 25), random_state=1, max_iter=700)

In [14]:
model_nn1.fit(nn_data_train, nn_labels_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(100, 25, 25), max_iter=700,
              random_state=1, solver='lbfgs')

In [15]:
nn_preds = model_nn1.predict(nn_data_test)

In [16]:
final_preds= []
for each_nn_prediction, final_test_data in zip(nn_preds,nn_data_test):
  if each_nn_prediction == 0:
    pred = model_rfc.predict(final_test_data)
    final_preds.append(pred)
  else:
    pred = model_svc.predict(final_test_data)
    final_preds.append(pred)

In [17]:
print(classification_report(final_labels_test_tf,final_preds))

              precision    recall  f1-score   support

        FAKE       0.95      0.97      0.96      2939
        REAL       0.97      0.94      0.96      2801

    accuracy                           0.96      5740
   macro avg       0.96      0.96      0.96      5740
weighted avg       0.96      0.96      0.96      5740



In [26]:
count_rfc = 0
count_nn = 0
count_svc = 0
for i in range(5740):
  if pred_rfc[i+len(pred_rfc) - 5740] == final_labels_test_tf[i] and pred_svc[i+len(pred_svc)-5740] != final_labels_test_tf[i]:
    count_rfc+=1
    if final_preds[i]!=final_labels_test_tf[i]:
      count_nn+=1
  if pred_rfc[i+len(pred_rfc) - 5740]!= final_labels_test_tf[i] and pred_svc[i+len(pred_svc) - 5740] == final_labels_test_tf[i]:
    count_svc+=1
    if final_preds[i]!=final_labels_test_tf[i]:
      count_nn+=1

print("Mis-classified by RFC: ", count_svc)
print("Mis-classified by SVC: ", count_rfc)
print("Mis-classification by RFC/SVC: ", count_rfc + count_svc)
rfc_svc_misclassification = count_rfc + count_svc
print("Mis-classified by meta-learning model: ", count_nn)

print("Rate of reduction in misclassification: ", ((rfc_svc_misclassification - count_nn) / rfc_svc_misclassification) * 100, "%")
#print(count_rfc, count_svc, count_nn)
#print(count_rfc + count_svc, count_nn)

Mis-classified by RFC:  149
Mis-classified by SVC:  80
Mis-classification by RFC/SVC:  229
Mis-classified by meta-learning model:  88
Rate of reduction in misclassification:  61.572052401746724 %
